In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.font_manager as font_manager
from matplotlib.ticker import MaxNLocator
from matplotlib.patches import Polygon,Circle


title_font = font_manager.FontProperties(family='Times New Roman', size=16, weight='normal')
label_font = font_manager.FontProperties(family='Times New Roman', size=16, weight='bold')
legend_font = font_manager.FontProperties(family='Times New Roman', size=14, weight='normal')
special_tick_size=15
size_marker=90

In [ ]:
def extend_line_to_axes(point1, point2, xlim, ylim):
    # If the x-coordinates of the two points are the same, the line is vertical to the x-axis
    if point1[0] == point2[0]:
        x = point1[0]
        y_bottom = ylim[0]
        y_top = ylim[1]
    else:
        # Calculate the slope and intercept of the line connecting the two points
        slope = (point2[1] - point1[1]) / (point2[0] - point1[0])
        intercept = point1[1] - slope * point1[0]

        # Calculate the two endpoints of the dashed line based on xlim
        x_left = xlim[0]
        y_left = slope * x_left + intercept

        x_right = xlim[1]
        y_right = slope * x_right + intercept

        # If the dashed line intersects the lower bound of ylim, take the point on the lower bound
        y_bottom = ylim[0] if y_left < ylim[0] else y_left

        # If the dashed line intersects the upper bound of ylim, take the point on the upper bound
        y_top = ylim[1] if y_right > ylim[1] else y_right

    return x_left, y_left, x_right, y_right


In [ ]:
#Energy Comsuption Calculation

E_elect=50*10**(-9) # 50 nJ/ bit
e_amp=100*10**(-12) #100 pJ/bit/m^2 the transmit amplifier 
bit_each_packet=2000 #2000 bit for each meassage
e_computation=5*10**(-9) # 5 nJ/ bit/ message
initial_capacity=0.5 # initial energy of sensor nodes
period_sendingMessage=1 # Sending each message during 1 day

#Calculate Energy Comsuption for Sending Message
def EnergySend(k,distance):
    return E_elect*k+e_amp*k*distance**2

#Calculate Energy Comsuption for Recieving Message
def EnergyRecieve(k):
    return E_elect*k

#Calculate Energy Comsuption for Fusing Message
def EnergyComputation(k):
    return e_computation*k
#Module 2:Calculate failure Rate

failure_rate_compare=10**(-3) #10^(-3) day^(-1)
def calFailureRate(capacity):
    return failure_rate_compare
def calTimeToFailure(failure_rate,period_sendingMessage):
    time_to_failure=-np.log(1-random.random())/failure_rate
    return time_to_failure

In [ ]:
def select_clusterHeads(living_nodes,round_last_head,P,round_count):
    clusterHead_list=[]
    for node in living_nodes:
        if(round_last_head[node]<(round_count-int(1/P))): #if this node have not been cluster-head in the last 1/P rounds
            threshold=P/(1-P*(round_count%int(1/P)))
            if(random.random()<threshold):
                clusterHead_list.append(node)
    return clusterHead_list

def distributeLivingNodesToHead(living_nodes,pos_list,clusterHead_list):
    head_map=dict()
    living_nodes_not_head=set(living_nodes)-set(clusterHead_list)
    for not_head_node in living_nodes_not_head:
        distance_square_to_Head_min=float('inf')
        Head_min=-1
        for head in clusterHead_list:
            distance_square_to_Head=(pos_list[not_head_node][0]-pos_list[head][0])**2+(pos_list[not_head_node][1]-pos_list[head][1])**2
            if(distance_square_to_Head<distance_square_to_Head_min):
                Head_min=head
                distance_square_to_Head_min=distance_square_to_Head
        head_map[not_head_node]=(Head_min,distance_square_to_Head_min)
    return head_map

def calEnergyConsumptionEach(living_nodes,clusterHead_list,pos_list,head_map,basic_information=2000):
    capacity_consumed=dict()
    living_nodes_not_head=set(living_nodes)-set(clusterHead_list)
    information_to_process_for_head=dict()
    if(clusterHead_list==[]):
        for node in living_nodes:
            capacity_consumed[node]=0
        return capacity_consumed
    for head in clusterHead_list:
        information_to_process_for_head[head]=0
    for node_not_head in living_nodes_not_head:
        Head_min,distance_square_to_Head_min=head_map[node_not_head]
        information_upload=basic_information
        information_to_process_for_head[Head_min]+=information_upload
        capacity_consumed[node_not_head]=EnergySend(information_upload,(distance_square_to_Head_min)**(1/2))
    for clusterHead in clusterHead_list:
        information=information_to_process_for_head[clusterHead]
        distance=np.sqrt(pos_list[clusterHead][0]**2+pos_list[clusterHead][1]**2)
        basic_information=2000
        capacity_consumed[clusterHead]=EnergyRecieve(information)+EnergyComputation(information)+EnergySend(basic_information,distance)
    return capacity_consumed

def CalRemainEnergyJudgeDeadAndUnsensoredNode(capacity_consumed,capacity_list,clusterHead_list,living_nodes,head_map):
    newly_dead_nodes=[] #dead nodes due to wear-out of battery
    unsensor_living_nodes=[] #Living nodes that can not connect with Base Station due to failure of Cluster-Head
    living_nodes_not_head=set(living_nodes)-set(clusterHead_list)
    for node in living_nodes:
        capacity_list[node]-=capacity_consumed[node]
        if(capacity_list[node]<0):
            newly_dead_nodes.append(node)
    for living_node_not_head in (set(living_nodes_not_head)-set(newly_dead_nodes)):
        Head_min,distance_square_to_Head_min=head_map[living_node_not_head]
        if Head_min in newly_dead_nodes:
            unsensor_living_nodes.append(living_node_not_head)
    return newly_dead_nodes,unsensor_living_nodes

In [ ]:
#Initialize the position of nodes
N=400
pos_list=[]
file_name='./structure_WSN/networkUsedInPaper/randomSquare_N='+str(N)+'.txt'
f=open(file_name,'r')
line=f.readline()
while(line!=''):
    for _ in range(len(line)):
        if(line[_]==' '):
            s=_
    x=float(line[:s])
    y=float(line[s:])
    pos_list.append([x,y])
    line=f.readline()
f.close()

capacity_mean_living_history=[]
living_num_history=[]
percentiles_show_list=list(np.arange(1,0.78,-0.01)) # we demonstrate snapshot of system when fraction of living nodes is 100%,99%,98%,...,1%
index_show_percentile=0 
#Simulation in one runs
capacity_list=np.ones(N)*initial_capacity #initialize capacity of all sensor nodes
living_nodes=set(list(range(N)))
lifetime_node=dict()

days_in_a_round=10 # update cluster-head for each 5 days
P=0.05 #fraction of cluster heads
round_last_head=np.ones(N)*(-2*int(1/P)) #the last round when node is cluster-head
round_count=0 #
day_count=0 #
radius_sensor=3

while(len(list(living_nodes))>0):
    clusterHead_list=select_clusterHeads(living_nodes,round_last_head,P,round_count)
    for clusterHead in clusterHead_list:
        round_last_head[clusterHead]=round_count
    head_map=distributeLivingNodesToHead(living_nodes,pos_list,clusterHead_list)
    
    percentile_live=len(list(living_nodes))/N#If , demonstrate
    if(abs(len(clusterHead_list)-0.05*len(list(living_nodes)))<100):
        if(percentile_live<=percentiles_show_list[index_show_percentile]):
            if(clusterHead_list==[]):
                continue
            percentile_show=str(percentiles_show_list[index_show_percentile])
            living_nodes_not_head=set(living_nodes)-set(clusterHead_list)
            set_DeadSensors=set(range(N))-set(living_nodes)
            points_clusterHead=[]
            points_livingSensorsNormal=[]
            points_deadSensors=[]
            for clusterHead in clusterHead_list:
                points_clusterHead.append(pos_list[clusterHead])
            for livingSensors in living_nodes_not_head:
                points_livingSensorsNormal.append(pos_list[livingSensors])
            for deadSensors in set_DeadSensors:
                points_deadSensors.append(pos_list[deadSensors])
            points_living=np.array(points_clusterHead+points_livingSensorsNormal)
            points_clusterHead,points_deadSensors,points_livingSensorsNormal,=np.array(points_clusterHead),np.array(points_deadSensors),np.array(points_livingSensorsNormal)

            ax = plt.gca()
            plt.xlim([-25, 25])
            plt.ylim([0, 50])

            if(len(clusterHead_list)==2):
                vec_two_point=[points_clusterHead[0][0]-points_clusterHead[1][0],points_clusterHead[0][1]-points_clusterHead[1][1]]
                mid_point=[(points_clusterHead[0][0]+points_clusterHead[1][0])/2,(points_clusterHead[0][1]+points_clusterHead[1][1])/2]
                vec_vertical=[vec_two_point[1],-vec_two_point[0]]
                another_point=np.array(mid_point)+np.array(vec_vertical)
                x_left, y_left, x_right, y_right = extend_line_to_axes(mid_point, another_point, [-25,25], [0,50])
                # Plot the dashed line
                plt.plot([x_left, x_right], [y_left, y_right], c='black', linestyle='--',linewidth=1)
            if(len(clusterHead_list)>2):
                vor = Voronoi(points_clusterHead)
                voronoi_plot_2d(vor,show_points=False,show_vertices=False,show_input=False,ax=ax,linewidth=1)

            plt.scatter(points_livingSensorsNormal[:, 0], points_livingSensorsNormal[:, 1],color='orange', marker='o',label='Living nodes',s=30)
            if(len(points_deadSensors)!=0):
                plt.scatter(points_deadSensors[:, 0], points_deadSensors[:, 1],color='red', marker='+',label='Failed nodes')
            else:
                plt.scatter(None,None,color='red', marker='+',label='Failed nodes')
            plt.scatter(points_clusterHead[:, 0], points_clusterHead[:, 1],facecolor='None',edgecolor='green', marker='*',s=200,label='Cluster heads')

            plt.scatter(0,0,color='black',marker='^',s=250)        
            plt.scatter(None,None,color='black',marker='^',s=50,label='Base Station')

            plt.xlim([-25, 25])
            plt.ylim([0, 50])
            plt.title(str(int(100*(1-len(list(living_nodes))/N)))+"% nodes failed",fontproperties=label_font)
            plt.xlabel("x coordinate",fontproperties=label_font)
            plt.ylabel("y coordinate",fontproperties=label_font)
            plt.legend(loc='upper left')
            #plt.savefig("./demo/N="+str(N)+"_"+str(int(100*(1-len(list(living_nodes))/N)))+"%failed.png",dpi=400,bbox_inches='tight')
            plt.show()
            index_show_percentile+=1
    if(index_show_percentile==len(percentiles_show_list)-1):
        break
    for day in range(days_in_a_round):
        capacity_consumed=calEnergyConsumptionEach(living_nodes,clusterHead_list,pos_list,head_map,basic_information=2000)
        newly_dead_nodes,unsensor_living_nodes=CalRemainEnergyJudgeDeadAndUnsensoredNode(capacity_consumed,capacity_list,clusterHead_list,living_nodes,head_map)
        for node in newly_dead_nodes:
            lifetime_node[node]=day_count
        living_nodes=living_nodes-set(newly_dead_nodes) # nodes die of battery wear-out
        day_count+=1
    node_to_failure=[]
    
    capacity_sum=0
    living_num=len(list(living_nodes))
    for node in living_nodes:
        capacity=capacity_list[node]
        capacity_sum+=capacity
        failure_rate=calFailureRate(capacity)
        period_sendingMessage=1
        failure_prob=1-np.exp(-failure_rate*period_sendingMessage)
        if(random.random()<failure_prob):
            node_to_failure.append(node)
            lifetime_node[node]=day_count
    living_num_history.append(living_num)
    capacity_mean_living_history.append(capacity_sum/len(living_nodes))
    living_nodes=living_nodes-set(node_to_failure) #nodes die of failure
    round_count+=1